### Run this in Python kernel

In [ ]:
%%html
<style>
body {
    font-family: "Arial", cursive, sans-serif; # font style of application
}
pre {
    font-family: "Courier New"; # font style of cell outputs
}
</style>  

### Run below in the backtesting_clojure kernel

In [1]:
; import libraries from kernel
(ns clojure-backtesting.demo
  (:require [clojure-backtesting.data :refer :all]
            [clojure-backtesting.data-management :refer :all]
            [clojure-backtesting.portfolio :refer :all]
            [clojure-backtesting.order :refer :all]
            [clojure-backtesting.evaluate :refer :all]
            [clojure-backtesting.plot :refer :all]
            [clojure-backtesting.counter :refer :all]
            [clojure-backtesting.automation :refer :all]
            [clojure-backtesting.parameters :refer :all]
            [clojure-backtesting.indicators :refer :all]
            [clojure-backtesting.direct :refer :all]
            [clojure.string :as str]
            [clojure.java.io :as io]
            [clojure.pprint :as pprint]
  ) ;; require all libriaries from core
)

nil

### Import dataset

In [2]:
; path to dataset = "/Volumes/T7/CRSP"
; change it to the relative path to your own dataset
;
(load-dataset "/Volumes/T7/CRSP" "main" add-aprc)

The dataset is already furnished by add-aprc. No more modification is needed.


"Date range: 1972-01-03 ~ 2017-02-10"

### Initialise portfolio （Go back here everytime you want to restart.）

In [3]:
;; initialise with current date and initial capital (= $10000)
(init-portfolio "1981-12-15" 1000);

In [4]:
(get-date)

"1981-12-15"

### Write a strategy

The following code implements a trading strategy called Golden Rule:

MA 15 cross above the MA 30 (golden cross)

MA 15 cross below the MA 30 (death cross)

So in the codes, MA15 and MA30 are compared on a daily basis, if golden cross occurs, then you set a buy order; if death cross occurs, then you set a sell order first 



Should increase the cache size first to reduce repeatitive File IO

In [8]:
(CHANGE-CACHE-SIZE 30)

#'clojure-backtesting.parameters/CACHE-SIZE

In [5]:
(while (< (compare (get-date) "1981-12-24") 0)
  ;; run for about a week
  (do
    (let [[MA15 MA30] [(moving-avg "14593" 15) (moving-avg "14593" 30)]]
      (if (> MA15 MA30)
        (order "14593" 0.1 :print false)
        (order "14593" 0 :remaining true))))
  (let [[MA15 MA30] [(moving-avg "25785" 15) (moving-avg "25785" 30)]]
    (if (> MA15 MA30)
      (order "25785" 1  :print false)
      (order "25785" 0 :remaining true)))
  (update-eval-report)
  (next-date))
(end-order)

true

In [6]:
;(pprint/print-table (take 200 (deref portfolio-value)))
(print-order-record 10)


|      :date |  :tic |  :price | :aprc | :quantity |
|------------+-------+---------+-------+-----------|
| 1981-12-16 | 14593 | 19.5625 | 22.43 |       0.1 |
| 1981-12-16 | 25785 |   17.25 | 55.94 |         1 |
| 1981-12-17 | 14593 | 21.1875 | 23.22 |       0.1 |
| 1981-12-17 | 25785 |  17.625 | 56.46 |         1 |
| 1981-12-18 | 14593 | 22.9375 | 24.03 |       0.1 |
| 1981-12-18 | 25785 |    18.0 | 56.98 |         1 |
| 1981-12-21 | 14593 | 21.9375 | 23.57 |       0.1 |
| 1981-12-21 | 25785 |    18.0 | 56.98 |         1 |
| 1981-12-22 | 14593 | 22.3125 | 23.75 |       0.1 |
| 1981-12-22 | 25785 |    17.5 | 56.29 |         1 |


nil

### Check portfolio record

In [7]:
;; view final portfolio
(print-portfolio)


| :asset | :price | :aprc | :quantity | :tot-val |
|--------+--------+-------+-----------+----------|
|   cash |    N/A |   N/A |       N/A |   996.42 |


nil

In [9]:
;; view portfolio value and return
(print-portfolio-record 10)


|      :date | :tot-value | :daily-ret | :tot-ret | :loan | :leverage | :margin |
|------------+------------+------------+----------+-------+-----------+---------|
| 1981-12-15 |   $1000.00 |      0.00% |    0.00% | $0.00 |      0.00 |   0.00% |
| 1981-12-16 |   $1000.00 |      0.00% |    0.00% | $0.00 |      0.00 |   0.00% |
| 1981-12-17 |   $1000.60 |     -0.00% |    0.03% | $0.00 |      0.00 |   0.00% |
| 1981-12-18 |   $1001.80 |      0.00% |    0.08% | $0.00 |      0.00 |   0.00% |
| 1981-12-21 |   $1001.67 |      0.00% |    0.07% | $0.00 |      0.00 |   0.00% |
| 1981-12-22 |    $998.96 |     -0.00% |   -0.05% | $0.00 |      0.00 |   0.00% |
| 1981-12-23 |    $995.33 |      0.00% |   -0.20% | $0.00 |      0.00 |   0.00% |
| 1981-12-24 |    $996.42 |      0.00% |   -0.16% | $0.00 |      0.00 |   0.00% |


nil

### Generate evaluation report

In [10]:
(print-eval-report 10)


|      :date | :tot-value |    :vol |  :r-vol |               :sharpe |             :r-sharpe | :pnl-pt | :max-drawdown |
|------------+------------+---------+---------+-----------------------+-----------------------+---------+---------------|
| 1981-12-16 |      $1000 | 0.0000% | 0.0000% |               0.0000% |               0.0000% |      $0 |        0.0000 |
| 1981-12-17 |      $1000 | 0.0000% | 0.0000% |   9419676466259.1450% |   9419676466259.1450% |      $0 |        0.0000 |
| 1981-12-18 |      $1001 | 0.0000% | 0.0000% |  32469602626935.1760% |  32469602626935.1760% |      $0 |        0.0000 |
| 1981-12-21 |      $1001 | 0.0000% | 0.0000% |  33522685497906.6330% |  33522685497906.6330% |      $0 |        0.0000 |
| 1981-12-22 |       $998 | 0.0000% | 0.0000% | -18079259030964.3000% | -18079259030964.3000% |      $0 |        0.0000 |
| 1981-12-23 |       $995 | 0.0000% | 0.0000% | -86461715180573.6600% | -86461715180573.6600% |      $0 |        0.0000 |
| 1981-12-24 |       $9

nil

### Plot variables

In [11]:
(def data (deref portfolio-value))

#'clojure-backtesting.demo/data

In [12]:
; Add legend name to series
(def data-to-plot
 (map #(assoc % :plot "portfolio")
  data))

#'clojure-backtesting.demo/data-to-plot

In [13]:
(first data-to-plot)

{:date "1981-12-15", :tot-value 1000, :daily-ret 0.0, :tot-ret 0.0, :loan 0.0, :leverage 0.0, :margin 0.0, :plot "portfolio"}

In [15]:
(plot data-to-plot :plot :date :daily-ret true)